In [ ]:
import numpy as np
import pandas as pd
import dask
import boto3
import os
import io
import dask.dataframe as dd
import s3fs
import pyarrow

set up credentials in file

In [ ]:
# insert your access key credentials
os.environ["AWS_ACCESS_KEY_ID"] = "xxxx"
os.environ["AWS_SECRET_ACCESS_KEY"] = "xxxx"


In [ ]:
# test readubg single pandas file to dask array
%time
s3 = boto3.client('s3')
bucket='datathon-2018'
key='raw/MelbDatathon2018/Samp_0/ScanOnTransaction/2015/Week27/QID3530815_20180713_20515_0.txt.gz'
obj = s3.get_object(Bucket=bucket, Key=key)
df = pd.read_csv(io.BytesIO(obj['Body'].read()),compression='gzip')

In [ ]:
%time
df.head()

In [ ]:
len(df) #368226

In [ ]:
# tests reading single parquet file to dask dataframe 
%time
df=dd.read_parquet('s3://datathon-2018/parquet/taps/2015_week27.parquet')
len(df) #1,287,268

In [ ]:
# test reading about 10 parquet files in to dask dataframe
df=dd.read_parquet('s3://datathon-2018/parquet/taps/2015_week2*.parquet')
len(df) # 5,666,495


In [ ]:
# test reading in all files to dask array
df=dd.read_parquet('s3://datathon-2018/parquet/taps/*.parquet')


In [ ]:
df.head()

In [ ]:
df=df.set_index('datetime')

In [ ]:
df=df.groupby(df.card_type).size().compute()

In [ ]:
# persist dataframe to cluster 
df = df.persist()
progress(df)

Read in  dataframe from parquet files in s3

In [ ]:
# test file
bucket_name='datathon-2018'

s3 = boto3.resource('s3')
bucket=s3.Bucket(bucket_name)
file_list_on=[]
file_list=[obj.key for obj in bucket.objects.filter(Prefix='parquet/taps') if obj.key.endswith(".parquet")]


In [ ]:
len(file_list)

In [ ]:
file_list[1]

In [ ]:
%%time
bucket_name='s3://datathon-2018/'
df_taps = pd.concat([pd.read_parquet(bucket_name+f) for f in file_list])

In [ ]:
len(df_taps)